# Common Lib

In [1]:
from alpha.notebook import *
import json

await init_notebook()

async def report(key:str, params:dict, result_df:pd.DataFrame):
    """key must be unique, contains a date"""
    await omicron.cache.sys.hmset_dict(key, params)
    
    html = f"""
    <html>
    <head>
        <style>
            table, th, td {{
                border-collapse: collapse;
                border: 0px;
                color: #666;
            }}
        </style>
    </head>
    <body>
    <h2 style="color:#c21">趋势线预测模型</h2>
    {build_table(result_df, "grey_light")}
    </body>
    </html>
    """
    send_html_email(f"趋势线预测模型 {key}", html)

# Strategy

In [5]:
async def scan(end:str, pred_threshold:float=0.05, win:int=30):
    """`win`日线预测涨幅排名靠前，20, 30，60多头排列,当日为上涨阳线"""
    end = arrow.get(end).date()
    nbars = 60
    start = tf.day_shift(end, -nbars + 1)
    
    results = []

    for code in Securities().choose(["stock"]):
        sec = Security(code)
        
        try:
            bars = await sec.load_bars(start, end, FrameType.DAY)
        except Exception:
            continue
            
        if len(bars) < nbars or np.count_nonzero(np.isfinite(bars["close"])) < nbars * 0.9:
            continue
            
        close = fillna(bars["close"].copy())
        c1, c0 = close[-2:]
        open_ = bars["open"][-1]

        sample_points = 15 if win <= 10 else 20
        preds, _ = predict_by_moving_average(close, win, 5, 3e-3, sample_points)
        if preds is None:
            continue
            
        pred = round(preds[-1]/c0 - 1,2)
        if pred < pred_threshold:
            continue
            
        row = [sec.display_name, sec.code, pred, open_, c0, round(c0/c1-1, 2)]
        
        for n in [5, 10, 20, 60]:
            adv_win = c0/close[-n] - 1
            row.append(round(adv_win,3))
            
        results.append(row)
        
    return pd.DataFrame(results, columns=["name", "code", "pred", "open", "close", "adv1", "adv5", "adv10", "adv20", "adv60"])

In [6]:
def query(df, filters:dict):
    df_ = df
    for win in filters.keys():
        if win == "isbull":
            continue
            
        adv_col = f"adv{win}"
        low, top = filters[win]
        df_ = df_[(df_[adv_col] <= top) & (df_[adv_col] >= low)]
        
    if filters.get("isbull", False):
        df_ = df_[df_.open <= df_.close]
        
    return df_.sort_values("pred", ascending=False)

In [8]:
end_dt = "2021-09-23"
pred_threshold = 0.1
win = 20
df = await scan(end_dt, pred_threshold, win)
filters = {
    1: (0,1),
    20:(0.2,1),
    60: (0.1, 0.5),
    "isbull": True
}

result = query(df, filters)

await report(f"scan.result.trendline.{end_dt}", {
    "end": end_dt,
    "pred_threshold": pred_threshold,
    "win": win,
    "filters": json.dumps(filters)
},result)

result

,name,code,pred,open,close,adv1,adv5,adv10,adv20,adv60
325,陕西煤业,601225.XSHG,0.23,16.100000,16.250000,0.03,0.033,0.075,0.419,0.474
330,中国外运,601598.XSHG,0.20,5.830000,5.900001,0.03,-0.003,0.035,0.204,0.201
375,麒盛科技,603610.XSHG,0.19,25.450001,26.000000,0.02,0.082,0.160,0.429,0.448
224,长春经开,600215.XSHG,0.19,8.270000,8.270000,0.00,0.042,0.147,0.286,0.109
99,太极股份,002368.XSHE,0.17,26.299999,27.370003,0.05,0.059,0.105,0.297,0.377
217,建发股份,600153.XSHG,0.17,8.790000,8.860000,0.02,-0.013,0.015,0.204,0.165
343,招商轮船,601872.XSHG,0.16,5.630000,5.790000,0.05,0.049,0.189,0.375,0.253
52,中银绒业,000982.XSHE,0.15,2.630000,2.780000,0.06,0.004,0.090,0.247,0.130
120,金禾实业,002597.XSHE,0.15,41.389996,42.810001,0.05,-0.027,-0.047,0.247,0.234
230,万通发展,600246.XSHG,0.15,11.150000,11.900000,0.07,0.040,0.206,0.303,0.319


In [ ]:
# visualize

In [ ]:
import os
save_to = "/tmp/trendline/0.3_0.5/"
os.makedirs(save_to, exist_ok=True)
for row in tmp.to_records():
    cs = Candlestick()
    await cs.plot(row["code"], "2021-09-22 15:00", save_to=save_to)

In [ ]:
tmp = query(df, {
    1: (0,1)
},True)
tmp[tmp.pred>0.15]